In [0]:
!pip install tensorflow==2.0.0-alpha0

     |████████████████████████████████| 79.9MB 1.2MB/s 
     |████████████████████████████████| 61kB 19.0MB/s 
     |████████████████████████████████| 3.0MB 32.0MB/s 
     |████████████████████████████████| 419kB 44.6MB/s 
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, MobileNetV2
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
#mount gdrive
from google.colab import drive

#check path to file
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**Params to be changed**

In [0]:
# Params to be changed

level = 6
model_path = 'gdrive/My Drive/adl_pro/models/'
model_name = 'level6_saurabh.h5'
root = 'gdrive/My Drive/adl_pro/data/'

**Model Architecture**

Design here, fit later

In [0]:
conv_base = InceptionV3(weights='imagenet',include_top=False, input_shape=(299, 299, 3))
conv_base.trainable = False

model = tf.keras.Sequential()
model.add(conv_base)
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(5,5), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(5,5)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

87916544/87910968 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 8, 8, 2048)        21802784  
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 8, 8, 128)         6553728   
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________

**No need to change anything below this.**

In [0]:
import os

cancer = root + 'level' + str(level) + '/train/cancer/'
healthy = root + 'level' + str(level) + '/train/healthy/'
count_cancer = len(os.listdir(cancer))
count_healthy = len(os.listdir(healthy))

print( 'No. of Cancer Files: ', count_cancer )
print( 'No. of Healthy Files: ', count_healthy )

No. of Cancer Files:  3383
No. of Healthy Files:  3942


**Run the cell below iff subsampled data is not available.**

samples = 260 for level 3

samples = 300 for level 4

samples = 400 for level 5

No sampling required for level 6

In [0]:
from shutil import copyfile
import os, random

dst = root + 'level' + str(level) + '/train/healthy_sub/'
if count_cancer < count_healthy:
  if not os.path.exists(dst):
    os.makedirs(dst)
  if len(os.listdir(dst)) == 0:
    for file in random.sample(os.listdir(healthy), count_cancer):
      copyfile(healthy+file, dst+file)


In [0]:
# open file in gdrive
augment_datagen = ImageDataGenerator( rescale = 1./255, rotation_range = 45,
                              width_shift_range = 0.2, height_shift_range = 0.2,
                              horizontal_flip = True, vertical_flip = True
                            )

if count_cancer < count_healthy:
  class_healthy = 'healthy_sub'
else:
  class_healthy = 'healthy'

train_generator = augment_datagen.flow_from_directory(
          root + 'level' + str(level) + '/train',
          target_size=(299, 299),
          batch_size=32,
          classes = ['cancer',class_healthy],
          class_mode='sparse')

normal_datagen = ImageDataGenerator( rescale = 1./255 )

validate_generator = normal_datagen.flow_from_directory(
          root + 'level' + str(level) + '/validate',
          target_size=(299, 299),
          batch_size=100,
          class_mode='sparse')

Found 6766 images belonging to 2 classes.
Found 1065 images belonging to 2 classes.


In [0]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=10,
        validation_data=validate_generator,
        #validation_steps=100
)

Epoch 1/10
10/10 [==============================] - 625s 63s/step - loss: 5.8793 - accuracy: 0.5000 - val_loss: 2.6214 - val_accuracy: 0.3737
Epoch 2/10
10/10 [==============================] - 409s 41s/step - loss: 2.2514 - accuracy: 0.5250 - val_loss: 1.3425 - val_accuracy: 0.3737
Epoch 3/10
10/10 [==============================] - 407s 41s/step - loss: 1.4048 - accuracy: 0.5464 - val_loss: 0.6553 - val_accuracy: 0.6272
Epoch 4/10
10/10 [==============================] - 412s 41s/step - loss: 1.0095 - accuracy: 0.4906 - val_loss: 0.6668 - val_accuracy: 0.6047
Epoch 5/10
10/10 [==============================] - 401s 40s/step - loss: 0.7228 - accuracy: 0.6250 - val_loss: 0.7643 - val_accuracy: 0.3746
Epoch 6/10
10/10 [==============================] - 401s 40s/step - loss: 0.6909 - accuracy: 0.5625 - val_loss: 0.6712 - val_accuracy: 0.6160
Epoch 7/10
10/10 [==============================] - 414s 41s/step - loss: 0.6502 - accuracy: 0.6250 - val_loss: 0.6837 - val_accuracy: 0.5615
Epoch 

In [0]:
model.save( model_path + model_name )